In [1]:
import numpy as np

In [2]:
class TreeNode2:

    def __init__(self, name, dims=3):
        self.name = name
        self.center = np.random.uniform(-1,1,size=(dims))
        self.radius =  1
        self.dims = dims
        self.left = None  #TreeNode
        self.right = None  #TreeNode
        self.label = None

    def run(self, inputs, indexes=None):
      if indexes is None: indexes = np.arange(len(inputs))
      self.indexes=indexes

      if self.right is None or self.left is None:
          return [self]
      else:
          norm = np.linalg.norm(inputs-self.center,axis=1)
          l_indexes = np.argwhere(norm < self.radius).flatten()
          r_indexes = list(set(range(len(inputs))) - set(l_indexes))
          r_ret = self.right.run(inputs[r_indexes,:],indexes[r_indexes])
          l_ret = self.left.run(inputs[l_indexes,:],indexes[l_indexes])
          return [self] + l_ret + r_ret
    
    def clone(self) :
        tree = TreeNode2(self.name)
        if self.right is not None : tree.right = self.right.clone()
        if self.left is not None : tree.left = self.left.clone()
        return tree
    
    def cut(self):
        self.right=None
        self.left=None
        
    def grow(self,depth):
        if depth>0:
          self.right = TreeNode2(str(self.name)+"R")
          self.right.grow(depth-1)
          self.left = TreeNode2(str(self.name)+"L")
          self.left.grow(depth-1)
    
    def show(self, space=0, tab_size=6):
        space += tab_size
        if self.right is not None: self.right.show(space, tab_size)
        s = str(self.name) + "(" + str(self.label) + ")"
        print(" "*(space-tab_size), s)
        if self.left is not None: self.left.show(space, tab_size)  
    
    def to_list(self):
        if self.right is None or self.left is None:
            return [self]
        else:
            return [self] + self.left.to_list() + self.right.to_list()
    
    def depth(self):
        if self.right is None or self.left is None:
            return 1
        else:
            return 1 + max(self.right.depth(),self.left.depth())
    

In [3]:
# Testing Class methods

In [4]:
tree1 = TreeNode2("-") #init
tree1.grow(2) #grow
tree1.show() #show

             -RR(None)
       -R(None)
             -RL(None)
 -(None)
             -LR(None)
       -L(None)
             -LL(None)


In [5]:
branch = tree1.right.clone() #clone
branch.show()

       -RR(None)
 -R(None)
       -RL(None)


In [6]:
tree1.right.cut() #cut
tree1.show()

       -R(None)
 -(None)
             -LR(None)
       -L(None)
             -LL(None)


In [7]:
nodes = tree1.to_list() #to_list
print(nodes)

[<__main__.TreeNode2 object at 0x0000023B4DB47748>, <__main__.TreeNode2 object at 0x0000023B4DB47908>, <__main__.TreeNode2 object at 0x0000023B4DB479E8>, <__main__.TreeNode2 object at 0x0000023B4DB478D0>, <__main__.TreeNode2 object at 0x0000023B4DB47710>]


In [8]:
inputs = np.random.uniform(-1,1,size=(20,3))
nodes = tree1.run(inputs) #run

for node in nodes:
    print(node.name, node.indexes)

- [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
-L [ 5 10 12 14]
-LL []
-LR [ 5 10 12 14]
-R [ 0  1  2  3  4  6  7  8  9 11 13 15 16 17 18 19]


In [9]:
#outside methods

In [10]:
def crossover(tree1,tree2):
    new_tree = tree1.clone()
    rnd_node_tree1 = np.random.choice(new_tree.to_list(),1)[0]
    subs_tree2 = np.random.choice(tree2.to_list(),2)
    rnd_node_tree1.right = subs_tree2[0]
    rnd_node_tree1.left = subs_tree2[1]    
    return new_tree     

In [11]:
tree1 = TreeNode2("1")
tree1.grow(2)
tree2 = TreeNode2("2")
tree2.grow(2)

tree3 = crossover(tree1,tree2)
tree3.show()

             1RR(None)
       1R(None)
             1RL(None)
 1(None)
                         2LR(None)
                   2L(None)
                         2LL(None)
             1LR(None)
                               2RR(None)
                         2R(None)
                               2RL(None)
                   2(None)
                               2LR(None)
                         2L(None)
                               2LL(None)
       1L(None)
             1LL(None)


In [12]:
def mutate(tree, m_rate=0.1):
    new_tree = tree.clone()
    for node in new_tree.to_list():
        if np.random.rand(1) < m_rate :
            node.center = np.random.uniform(-1,1,size=(node.dims))
            node.radius =  np.random.rand(1)
    return new_tree    

In [14]:
nodes = tree3.run(inputs) #run

for node in nodes:
    print(node.name, node.indexes)
    
tree4 = mutate(tree3)

nodes = tree4.run(inputs) #run

for node in nodes:
    print(node.name, node.indexes)

1 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
1L [ 0  9 12 13 14 15 16]
1LL []
1LR [ 0  9 12 13 14 15 16]
2 [13 14]
2L []
2LL []
2LR []
2R [13 14]
2RL []
2RR [13 14]
2L []
2LL []
2LR []
1R [ 1  2  3  4  5  6  7  8 10 11 17 18 19]
1RL [ 1  7 11]
1RR [ 2  3  4  5  6  8 10 17 18 19]
1 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
1L [ 1  7 11]
1LL [ 1 11]
1LR [7]
2 []
2L []
2LL []
2LR []
2R []
2RL []
2RR []
2L [7]
2LL []
2LR [7]
1R [ 0  2  3  4  5  6  8  9 10 12 13 14 15 16 17 18 19]
1RL [ 3 12 13 14 19]
1RR [ 0  2  4  5  6  8  9 10 15 16 17 18]
